# Загрузка библиотек

In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from psycopg2.extras import execute_values
from faker import Faker
from datetime import datetime, timedelta
import random
from typing import List, Tuple

# Генерация данных

In [30]:
conn = psycopg2.connect(dbname="Farpost", user="postgres", password="86754231qaZ", host="localhost", port="5434")

In [11]:
conn.close()

In [14]:
def insert_users(users: List[Tuple]):
    try:
        sql = """INSERT INTO "Users" (username, password_hash, email, created_at)
        VALUES %s"""
        cur = conn.cursor()

        execute_values(cur, sql, users)
        conn.commit()
    except Exception as e:
        print("Ошибка:", e)
        conn.close()

In [15]:
def insert_actions(actions: List[Tuple]):
    try:
        sql = """INSERT INTO "Actions" (type)
        VALUES %s"""
        cur = conn.cursor()
        execute_values(cur, sql, actions)
        conn.commit()

    except Exception as e:
        print("Ошибка:", e)
        conn.close()

In [16]:
faker = Faker()

users = [(
    faker.user_name(),
    faker.sha256(),
    faker.email(),
    datetime.now() - timedelta(days=random.randint(90, 180)))
    for i in range(10)
    ]

annonimus = [(None, None, None, None)
    for i in range(10)]

users = users + annonimus

insert_users(users)

In [17]:
actions = [
    ("Первый заход на сайт",),
    ("Регистрация",),
    ("Логин",),
    ("Логаут",),
    ("Создание темы",),
    ("Заход на тему",),
    ("Удаление темы",),
    ("Написание сообщения",)
]

insert_actions(actions)

In [18]:
def get_action_id(action):
    action_id = pd.read_sql(f'''
    SELECT action_id
    FROM "Actions"
    WHERE type = \'{action}\'''', conn)["action_id"].tolist()[0]

    return action_id

In [19]:
def user_is_logined(user_id: int):
    user_info = pd.read_sql(f'''
    SELECT * 
    FROM "Users"
    WHERE user_id = {user_id}''', conn)["username"].tolist()[0]

    return user_info is not None

In [20]:
def insert_topic(topic: Tuple):
    try:
        sql = """INSERT INTO "Content" (type, created_at)
        VALUES (%s, %s)
        RETURNING content_id;"""
        cur = conn.cursor()
        cur.execute(sql, ("Тема", topic[1]))

        content_id = cur.fetchone()[0]

        sql = """INSERT INTO "Topics"
        VALUES (%s, %s);"""
        cur.execute(sql, (content_id, topic[0]))
        
        conn.commit()

        return content_id
    except Exception as e:
        print("Ошибка:", e)
        conn.close()

In [21]:
def insert_message(message: Tuple):
    try:
        sql = """INSERT INTO "Content" (type, created_at)
        VALUES (%s, %s)
        RETURNING content_id;"""
        cur = conn.cursor()
        cur.execute(sql, ("Сообщение", message[2]))

        content_id = cur.fetchone()[0]

        sql = """INSERT INTO "Messages"
        VALUES (%s, %s, %s);"""
        cur.execute(sql, (content_id, message[1], message[0]))
        
        conn.commit()

        return content_id
    except Exception as e:
        print("Ошибка:", e)
        conn.close()

In [22]:
def insert_log(log: Tuple):
    try:
        sql = """INSERT INTO "Logs" (user_id, action_id, is_success, created_at)
        VALUES (%s, %s, %s, %s)
        RETURNING log_id;"""
        cur = conn.cursor()
        cur.execute(sql, log)

        log_id = cur.fetchone()[0]
        
        conn.commit()

        return log_id
    except Exception as e:
        print("Ошибка:", e)
        conn.close()

In [23]:
def insert_content_log(log_id: int, content_id: int):
    try:
        sql = """INSERT INTO "Content_Log" (log_id, content_id)
        VALUES (%s, %s);"""
        cur = conn.cursor()
        cur.execute(sql, (log_id, content_id))
        conn.commit()
        
    except Exception as e:
        print("Ошибка:", e)
        conn.close()

In [24]:
actions = pd.read_sql('SELECT * FROM "Actions"', conn)["type"].tolist()
users_id = pd.read_sql('SELECT user_id FROM "Users"', conn)["user_id"].tolist()

for day in range(1, 32):
    created_at = datetime(year=2025, month=1, day=day)

    for action in actions:
        action_id = get_action_id(action)
        num_actions = random.choice(range(5, 11))

        for i in range(num_actions):
            user_id = random.choice(users_id)
            if action == "Создание темы":
                if user_is_logined(user_id):
                    is_success = True
                    topic_title = faker.sentence()
                    content_id = insert_topic((topic_title, created_at))
                    log_id = insert_log((user_id, action_id, is_success, created_at))
                    insert_content_log(log_id, content_id)
                else:
                    is_success = False
                    insert_log((user_id, action_id, is_success, created_at))
            
            elif action == "Написание сообщения":
                is_success = True
                topics_id = pd.read_sql('SELECT content_id FROM "Content" WHERE type = \'Тема\'', conn)["content_id"].tolist()
                topic_id = random.choice(topics_id)
                message_content = faker.sentence()
                content_id = insert_message((message_content, topic_id, created_at))
                log_id = insert_log((user_id, action_id, is_success, created_at))
                insert_content_log(log_id, content_id)
            
            else:
                is_success = True
                insert_log((user_id, action_id, is_success, created_at))

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/3130861170.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  actions = pd.read_sql('SELECT * FROM "Actions"', conn)["type"].tolist()
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/3130861170.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users_id = pd.read_sql('SELECT user_id FROM "Users"', conn)["user_id"].tolist()
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/3130861170.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlc

In [25]:
conn.close()

In [29]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_values
from faker import Faker
from datetime import datetime, timedelta
import random
from typing import List, Tuple



conn = psycopg2.connect(dbname="Farpost", user="postgres", password="86754231qaZ", host="localhost", port="5434")

faker = Faker()


def insert_users(users: List[Tuple]):
    try:
        sql = """INSERT INTO "Users" (username, password_hash, email, created_at)
        VALUES %s"""
        cur = conn.cursor()

        execute_values(cur, sql, users)
        conn.commit()
    except Exception as e:
        print("Ошибка:", e)
        conn.close()


def insert_actions(actions: List[Tuple]):
    try:
        sql = """INSERT INTO "Actions" (type)
        VALUES %s"""
        cur = conn.cursor()
        execute_values(cur, sql, actions)
        conn.commit()

    except Exception as e:
        print("Ошибка:", e)
        conn.close()


def generate_users(created_at):
    num_users = random.choice(range(5, 11))

    users = [(
        faker.user_name(),
        faker.sha256(),
        faker.email(),
        created_at)
        for i in range(num_users)
        ]

    annonimus = [(None, None, None, created_at)
        for i in range(num_users)]

    users = users + annonimus

    insert_users(users)


actions = [
    ("Первый заход на сайт",),
    ("Регистрация",),
    ("Логин",),
    ("Логаут",),
    ("Создание темы",),
    ("Заход на тему",),
    ("Удаление темы",),
    ("Написание сообщения",)
]

insert_actions(actions)


def get_action_id(action):
    action_id = pd.read_sql(f'''
    SELECT action_id
    FROM "Actions"
    WHERE type = \'{action}\'''', conn)["action_id"].tolist()[0]

    return action_id


def user_is_logined(user_id: int):
    user_info = pd.read_sql(f'''
    SELECT * 
    FROM "Users"
    WHERE user_id = {user_id}''', conn)["username"].tolist()[0]

    return user_info is not None


def insert_topic(topic: Tuple):
    try:
        sql = """INSERT INTO "Content" (type, created_at)
        VALUES (%s, %s)
        RETURNING content_id;"""
        cur = conn.cursor()
        cur.execute(sql, ("Тема", topic[1]))

        content_id = cur.fetchone()[0]

        sql = """INSERT INTO "Topics"
        VALUES (%s, %s);"""
        cur.execute(sql, (content_id, topic[0]))
        
        conn.commit()

        return content_id
    except Exception as e:
        print("Ошибка:", e)
        conn.close()


def insert_message(message: Tuple):
    try:
        sql = """INSERT INTO "Content" (type, created_at)
        VALUES (%s, %s)
        RETURNING content_id;"""
        cur = conn.cursor()
        cur.execute(sql, ("Сообщение", message[2]))

        content_id = cur.fetchone()[0]

        sql = """INSERT INTO "Messages"
        VALUES (%s, %s, %s);"""
        cur.execute(sql, (content_id, message[1], message[0]))
        
        conn.commit()

        return content_id
    except Exception as e:
        print("Ошибка:", e)
        conn.close()


def insert_log(log: Tuple):
    try:
        sql = """INSERT INTO "Logs" (user_id, action_id, is_success, created_at)
        VALUES (%s, %s, %s, %s)
        RETURNING log_id;"""
        cur = conn.cursor()
        cur.execute(sql, log)

        log_id = cur.fetchone()[0]
        
        conn.commit()

        return log_id
    except Exception as e:
        print("Ошибка:", e)
        conn.close()


def insert_content_log(log_id: int, content_id: int):
    try:
        sql = """INSERT INTO "Content_Log" (log_id, content_id)
        VALUES (%s, %s);"""
        cur = conn.cursor()
        cur.execute(sql, (log_id, content_id))
        conn.commit()
        
    except Exception as e:
        print("Ошибка:", e)
        conn.close()


actions = pd.read_sql('SELECT * FROM "Actions"', conn)["type"].tolist()

for day in range(1, 32):
    created_at = datetime(year=2025, month=1, day=day)

    generate_users(created_at)

    users_id = pd.read_sql('SELECT user_id FROM "Users"', conn)["user_id"].tolist()

    for action in actions:
        action_id = get_action_id(action)
        num_actions = random.choice(range(5, 11))

        for i in range(num_actions):
            user_id = random.choice(users_id)
            if action == "Создание темы":
                if user_is_logined(user_id):
                    is_success = True
                    topic_title = faker.sentence()
                    content_id = insert_topic((topic_title, created_at))
                    log_id = insert_log((user_id, action_id, is_success, created_at))
                    insert_content_log(log_id, content_id)
                else:
                    is_success = False
                    insert_log((user_id, action_id, is_success, created_at))
            
            elif action == "Написание сообщения":
                is_success = True
                topics_id = pd.read_sql('SELECT content_id FROM "Content" WHERE type = \'Тема\'', conn)["content_id"].tolist()
                topic_id = random.choice(topics_id)
                message_content = faker.sentence()
                content_id = insert_message((message_content, topic_id, created_at))
                log_id = insert_log((user_id, action_id, is_success, created_at))
                insert_content_log(log_id, content_id)
            
            else:
                is_success = True
                insert_log((user_id, action_id, is_success, created_at))


conn.close()

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2077065527.py:169: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  actions = pd.read_sql('SELECT * FROM "Actions"', conn)["type"].tolist()
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2077065527.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users_id = pd.read_sql('SELECT user_id FROM "Users"', conn)["user_id"].tolist()
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2077065527.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQ

## Агрегационные функции

In [34]:
def get_new_accounts_count():
    new_accounts_count = pd.read_sql('''
    SELECT EXTRACT(DAY FROM created_at) AS p_day, COUNT(*)
    FROM "Users"
    GROUP BY p_day
    ORDER BY p_day;''', con=conn)

    return new_accounts_count

In [35]:
get_new_accounts_count()

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2820772007.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_accounts_count = pd.read_sql('''


,p_day,count
0,1.0,20
1,2.0,14
2,3.0,20
3,4.0,16
4,5.0,14
5,6.0,18
6,7.0,10
7,8.0,12
8,9.0,20
9,10.0,14


In [54]:
def get_message_anonymous_percentage(day: int):
    message_users = pd.read_sql(f'''
    SELECT u.*
    FROM "Content" AS co
    JOIN "Content_Log" AS cl
        ON cl.content_id = co.content_id
    JOIN "Logs" AS l
        ON l.log_id = cl.log_id
    JOIN "Users" AS u
        ON l.user_id = u.user_id
    WHERE co.type = 'Сообщение' AND EXTRACT(day FROM co.created_at) = {day};''', con=conn)
    
    percentage = round(len(message_users[message_users["username"].isnull()]) / len(message_users) * 100, 1)

    return percentage

In [55]:
get_message_anonymous_percentage(3)

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/478359369.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  message_users = pd.read_sql(f'''


85.7

In [56]:
def get_new_messages_count():
    new_messages_count = pd.read_sql('''
    SELECT EXTRACT(DAY FROM created_at) AS p_day, COUNT(*)
    FROM "Content" co
    WHERE co.type = 'Сообщение'
    GROUP BY p_day
    ORDER BY p_day;''', con=conn)

    return new_messages_count

In [57]:
get_new_messages_count()

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/370767232.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_messages_count = pd.read_sql('''


,p_day,count
0,1.0,9
1,2.0,8
2,3.0,7
3,4.0,10
4,5.0,6
5,6.0,10
6,7.0,10
7,8.0,6
8,9.0,5
9,10.0,5


In [63]:
def get_new_topics_percentage(day: int):
    prev_days_topics_count = pd.read_sql(f'''
    SELECT COUNT(*)
    FROM "Content" co
    WHERE co.type = 'Тема' AND EXTRACT(DAY FROM created_at) < {day};''', con=conn)

    cur_day_topics_count = pd.read_sql(f'''
    SELECT COUNT(*)
    FROM "Content" co
    WHERE co.type = 'Тема' AND EXTRACT(DAY FROM created_at) = {day};''', con=conn)

    percentage = round(cur_day_topics_count["count"].iloc[0] / prev_days_topics_count["count"].iloc[0] * 100, 1)

    return percentage

In [66]:
get_new_topics_percentage(30)

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2926770589.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prev_days_topics_count = pd.read_sql(f'''
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2926770589.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cur_day_topics_count = pd.read_sql(f'''


3.0

In [72]:
data = pd.DataFrame({"day": range(1, 32)})

data["new_accounts"] = get_new_accounts_count()["count"]
data["new_messages"] = get_new_messages_count()["count"]
data["anonymous_messages_pct"] = data["day"].apply(get_message_anonymous_percentage)
data["new_topics_pct"] = data["day"].apply(get_new_topics_percentage)

/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/2820772007.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_accounts_count = pd.read_sql('''
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/370767232.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_messages_count = pd.read_sql('''
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm0000gn/T/ipykernel_15977/478359369.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  message_users = pd.read_sql(f'''
/var/folders/lq/13c4rlw92f7_q2025mqwt1fm

In [74]:
data

,day,new_accounts,new_messages,anonymous_messages_pct,new_topics_pct
0,1,20,9,66.7,inf
1,2,14,8,50.0,120.0
2,3,20,7,85.7,9.1
3,4,16,10,50.0,33.3
4,5,14,6,50.0,31.2
5,6,18,10,40.0,9.5
6,7,10,10,30.0,17.4
7,8,12,6,50.0,7.4
8,9,20,5,20.0,3.4
9,10,14,5,80.0,10.0
